In [ ]:
# 多次输出结果（正确结果），互相比对，选择重复度最高的
# 与code文件夹平行位置有个文件env用来存放api_key

In [ ]:
import streamlit as st
from openai import OpenAI, OpenAIError
import time
import base64

import os
from dotenv import load_dotenv
from pathlib import Path

# 设置 API 密钥（省着点用。。）
# api_key = ' '
# openai.api_base = 'https://api.gpts.vin'
# openai.api_key = ' '
# 初始化 OpenAI 客户端
# client = OpenAI(api_key=api_key)
env_path = Path(__file__).resolve().parent.parent / ".env"
load_dotenv(dotenv_path=env_path)

client = OpenAI(
    base_url="https://one-api.guanzhao12.com/v1",
    api_key=os.getenv("API_KEY")
)

In [ ]:
# import streamlit as st
# from openai import OpenAI, OpenAIError
# import time
# import base64

# # 设置 API 密钥（购买于淘宝）
# client = OpenAI(
#     base_url="https://apikeyplus.com/v1",
#     api_key=" "
# )


In [4]:
# # 测试案例

# prompt="帮我写一个一百字作文"
# response = client.chat.completions.create(
#     # model="gpt-4o",
#     model="gpt-3.5-turbo",
#     messages=[
#         {"role": "system", "content": "You are a helpful assistant."},
#         {
#             "role": "user",
#             "content": [
#                 {"type": "text", "text": prompt}
#             ],
#         },  # 将图片数据作为二进制格式传递
#     ],
#     max_tokens = 2000,
#     temperature = 0.5
# )
# print(response.choices[0].message.content)
# # print(response)

In [5]:
# 根据提示词、附件图片和温度，返回GPT生成文本
# 设置默认温度为0.5；温度可在0-1间调整，越小则回答越保守，越大则回答越具创造性
def generateGPTtext(prompt, file_path = "NONE", temp = 0.5): 
    # 如果仅有提示词
    if file_path == "NONE": 
        # 发送文本生成请求
        try:
            response = client.chat.completions.create(
                model="gpt-4o",
                # model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": "You are a helpful assistant."},
                    {
                        "role": "user",
                        "content": [
                            {"type": "text", "text": prompt}
                        ],
                    },  # 将图片数据作为二进制格式传递
                ],
                max_tokens = 2000,
                temperature = temp
            )
        except OpenAIError as e:
            # OpenAI API 的一般错误
            print("OpenAI API 错误")
            print(e)
        except Exception as e:
            # 其他错误
            print("错误")
            print(e)
            
    # 如果有提示词和附件图片
    else:
        # 读取并编码图片
        try:
            with open(file_path, "rb") as image_file:
                image_data = image_file.read()
                image_base64 = base64.b64encode(image_data).decode()
        except FileNotFoundError:
            print(f"文件 '{file_path}' 不存在。")
            exit(1)
        # 发送文本生成请求
        try:
            response = client.chat.completions.create(
                model="gpt-4o",
                # model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": "You are a helpful assistant."},
                    {
                        "role": "user",
                        "content": [
                            {"type": "text", "text": prompt},
                            {
                                "type": "image_url",
                                "image_url": {"url": f"data:image/jpeg;base64,{image_base64}"},
                            },
                        ],
                    },
                ],
                max_tokens = 2000,
                temperature = temp
            )
        except OpenAIError as e:
            # OpenAI API 的一般错误
            print("OpenAI API 错误")
            print(e)
        except Exception as e:
            # 其他错误
            print("错误")
            print(e)
            
    return response.choices[0].message.content

In [6]:
# 截取code_string中代码的部分，并根据code_index命名后保存到本地
def get_python_code(code_string, code_index):
    import re
    import os
    # 使用正则表达式提取 Python 代码块
    match = re.search(r'```python\n(.*?)\n```', code_string, re.DOTALL)
    
    if match:
        python_code = match.group(1)
        code_name = 'code_' + str(code_index) + '.py'
        save_path = 'F:\\QuantData\\factor_pycode'
        # 构建完整的文件路径
        full_path = os.path.join(save_path, code_name)
         # 保存为 .py 文件，使用 UTF-8 编码，指定命名和位置
        with open(full_path, 'w', encoding='utf-8') as file:
            file.write(python_code)

        return True
        
    else:
        print("未找到 Python 代码块")
        return False

In [7]:
# 根据上次生成的代码以及产生的报错信息，生成新的代码
def regenerate_code(code_string, code_index, error_info):
    print(error_info)
    prompt = str(code_string) + "\n 以上代码运行有误，请根据以下报错信息，重新生成代码：\n" + str(error_info)
    
    code_string = generateGPTtext(prompt)

    # 更新code_string在本地的txt文件
    file_path = 'F:\\QuantData\\factor_code_string\\error_update_' + str(code_index) + '.txt' 
    
    with open(file_path, "w", encoding="utf-8") as file:
        file.write(code_string)
    
    # 截取的代码文件会直接保存到本地等待之后调用，因此无需作为函数返回值
    get_python_code(code_string, code_index) 

In [8]:
# 根据因子图片给出对因子的初步分析
def step1(file_path):

    prompt = "图片介绍了一个用于评价股票的因子，请详细拆分并一步一步给出这个因子的计算方法，以及因子的含义与逻辑" + \
            "\n输出内容使用Latex格式，能在Overleaf中通过编译)"
    
    # file_path = 'F:\\QuantData\\image\\factor\\factor.png' # 选择图片文件的路径
    # file_path = 'F:\\QuantData\\image\\factor\\factor1.jpg'
    
    factor_analysis = generateGPTtext(prompt, file_path)
    
    return factor_analysis
    
    # print(factor_analysis)

In [9]:
# 提供具体df，生成计算因子的Python代码
def step2(factor_analysis, code_index):

    prompt = factor_analysis + \
    "需要处理的股票数据共有以下这些列：SecuCode: 股票代码；TradeDay: 交易日期；TradeTime: 交易时间（精确到分钟），\
    每天上午09:30:00-11:30:00，下午13:00:00-15:00:00，其他时间不交易；PreClosePrice: 前一分钟收盘价；OpenPrice: 开盘价；HighPrice: 最高价；LowPrice: 最低价；\
    ClosePrice: 收盘价;Amount: 成交金额;Volume: 成交量;TradesCount: 成交笔数。\
    原文件储存在'F:\\QuantData\\minute_data'，文件名为'202406_oneminute.feather'（请不要尝试读取原文件，因为可能数据量很大）。\
    根据这个数据表，先分析上面的因子是否可以由该数据表的列信息计算得到，若否，给出不能计算的原因，只要存在无法计算的元素，就不要输出任何代码；若是，给出上面因子计算的Python代码" + \
    "其中命令包含从本地读取指定数据文件，最后生成数据表仅包含'SecuCode','TradeDay'和所求得的因子值这三列，并以feather格式保存到本地\
    （F:\\QuantData\\factor_result），命名为'df_'" + str(code_index) + \
    "代码风格要求：（1）尽可能对原数据表进行列操作，计算过程中产生的中间变量可视情况合并到原数据表中，用于后面的计算；（2）尽可能减少使用agg，apply；" \
    + "（3）在开头包含import numpy as np, 且不要包含import feather" \
    + "（4）打印保存代码的路径"
    
    file_path = 'F:\\QuantData\\image\\df.png' # 股票数据图片文件的路径
    
    code_string = generateGPTtext(prompt, file_path, 0.5) # 设置低温度，为了使回答尽可能严谨（尤其是当无法计算时，要忠实地给出结论）

    # code_string保存为txt文件，便于检查
    file_path = 'F:\\QuantData\\factor_code_string\\' + str(i) + '.txt' 
    
    with open(file_path, "w", encoding="utf-8") as file:
        file.write(code_string)

    # 检验code_string中是否包含 Python 代码，若否，则说明该因子无法通过现有数据计算得到
    if_code = get_python_code(code_string, code_index)

    return code_string, if_code

In [10]:
# 运行上面分析得到的代码，如果报错，则根据错误信息重新生成代码，直到运行成功
def step3(code_string, code_index):

    import subprocess
    
    # 从指定路径获取指定脚本文件
    script_path = 'F:\\QuantData\\factor_pycode\\'
    script_name = script_path + 'code_' + str(code_index) + '.py'
    # 记录重复运行次数（有报错时）
    try_count = 1
    # 用于累计历史报错信息
    all_error_info = ''
    
    while try_count < 4:
        try:
            # 使用 subprocess.run 执行脚本并捕获输出
            result = subprocess.run(
                ['python', script_name],   # 执行的命令
                capture_output=True,        # 捕获标准输出和标准错误
                text=True,                  # 将输出解码为字符串
                check=True                  # 如果脚本返回非零状态码则抛出异常
            )
            # 打印脚本的标准输出
            print("标准输出:\n", result.stdout)
            return result.stdout
            break # 若生成的代码运行成功，则退出
            
        except subprocess.CalledProcessError as e:
            # 打印脚本的标准错误 (具体错误内容)
            # print("标准错误:\n", e.stderr)
            print("标准错误:")
            
            # 打印错误代码
            print("错误代码:", e.returncode)
    
            print("重复运行次数：", try_count)
            try_count = try_count + 1

            # 将历史报错信息进行累加
            all_error_info = all_error_info + e.stderr + '\n'
            # 重新生成代码
            regenerate_code(code_string, code_index, all_error_info)
            
        except Exception as e:
            # 处理其他可能的异常
            print("发生异常:\n", e)
    
            break
            
    # 如果连续报错以及连续生成代码超过三次，则放弃这个因子
    return "ERROR"

In [11]:
# 获取最终结果数据文件的路径
def step4(path_text):
    # 显示最终得到的结果数据
    prompt = path_text + \
    "请直接给出路径，用双斜杠线，不要带任何其他内容"
    
    result_path = generateGPTtext(prompt, "NONE", 0)
    
    print(result_path)
    
    return result_path

In [12]:
# # 识别因子图片，结果保存为txt文件

# start_idx = 123  # 准备测试因子序列的首个
# end_idx = 232   # 准备测试因子序列的末个

# for i in range(start_idx, end_idx + 1): # 选择需要处理的因子
#     # factor_reliable = True # 初始默认该因子可靠
#     factor_image_path = 'F:\\QuantData\\factor_image\\' + str(i) + '.png' # 因子图片文件的路径
    
#     # 根据因子图片给出对因子的初步分析
#     step1_ans = step1(factor_image_path)

#     # 因子的初步分析保存为txt文件，便于后面进一步修改和调整
#     file_path = 'F:\\QuantData\\factor_image_analysis\\' + str(i) + '.txt' 
    
#     with open(file_path, "w", encoding="utf-8") as file:
#         file.write(step1_ans)

In [13]:
import pandas as pd
import itertools

factor_reliable_list = [] # 记录每个因子的可靠情况，第一列表示因子序号，第二列为True或者False

start_idx = 109  # 准备测试因子序列的首个
end_idx = 109    # 准备测试因子序列的末个
gp = 2          # 测试轮次
rt = 3          # 同一因子测试次数

for i in range(start_idx, end_idx + 1):
    factor_reliable = True # 初始默认该因子可靠

    # # 直接读取因子图片
    # factor_image_path = 'F:\\QuantData\\factor_image\\' + str(i) + '.jpeg'
    # # 根据因子图片给出对因子的初步分析
    # step1_ans = step1(factor_image_path)

    # 已有识别因子图片后得到的txt文件，读取txt文件
    factor_txt_path = 'F:\\QuantData\\factor_image_analysis\\' + str(i) + '.txt' 
    
    with open(factor_txt_path, "r", encoding="utf-8") as file:
        step1_ans = file.read()

    
    dfs = []
    for j in range(1, rt + 1): # 同一因子连续跑出rt个结果
        code_index = str(10000 * gp + i) + '_' + str(j)
        
        # 提供具体df，生成计算因子的Python代码
        step2_ans, if_code = step2(step1_ans, code_index)

        if if_code: # 如果存在 Python 代码，则运行上面分析得到的代码，如果报错，则根据错误信息重新生成代码，直到运行成功
            
            step3_ans = step3(step2_ans, code_index) 
            if step3_ans == 'ERROR':
                print("第"+str(i)+"个因子"+"结果不可靠,因为多次生成代码仍然报错\n")
                factor_reliable = False
                break
        else: # 如果不存在 Python 代码，则直接宣布该因子无效
            print("第"+str(i)+"个因子"+"结果不可靠,因为该因子无法通过现有数据计算得到\n")
            factor_reliable = False
            break
        
        # 获取最终结果数据文件的路径
        step4_ans = step4(step3_ans)
    
        df = pd.read_feather(step4_ans)

        # 判断df是否满足格式要求：恰有三列数据；列名依次为'SecuCode''TradeDay'和因子值；因子值有一半以上不是NaN；没有重复行
        # 如果不满足，需要重新生成 Python 代码
        try_count = 1
        while not (df.shape[1] == 3 and list(df.columns[:2]) == ['SecuCode', 'TradeDay'] and not df.duplicated(subset=['TradeDay', 'SecuCode']).any() and df.iloc[:, -1].isna().sum() < len(df) / 2 and df.iloc[:, -1].eq(0).sum() < len(df) / 2):
            
            if not (df.shape[1] == 3 and list(df.columns[:2]) == ['SecuCode', 'TradeDay']):
                print("列数与列名不符")
            elif df.duplicated(subset=['TradeDay', 'SecuCode']).any():
                print("重复索引")
            else:
                print("NaN太多或0太多")
            
            if try_count == 3:
                try_count = try_count + 1
                break
            else:
                print("df格式不满足要求，重新生成代码")
                step2_ans = step2(step1_ans, code_index)
                step3_ans = step3(step2_ans, code_index)
                step4_ans = step4(step3_ans)
                df = pd.read_feather(step4_ans)
                try_count = try_count + 1
                
        if try_count == 4:
            print("第"+str(i)+"个因子"+"结果不可靠,因为多次生成数据仍不满足格式要求\n")
            factor_reliable = False
            break

        print(df)
        print('含有NaN数量：', df.iloc[:, -1].isna().sum())
        dfs.append(df) # dfs中保存多次运行成功得到的因子列，用于之后的相关性分析

    # 如果前面已经得出因子不可靠的结论，则直接进行下一个因子的处理
    if factor_reliable == False:
        factor_reliable_list.append([i, False])
        continue


    # 对同一因子的多个计算结果进行相关性分析
    df_corr = dfs[0].set_index(['TradeDay','SecuCode'])
    df_corr = df_corr.rename(columns={df_corr.columns[-1]: 'result_1'})
    
    for k in range(1, rt):
        df_corr['result_' + str(k + 1)] = dfs[k].set_index(['TradeDay','SecuCode']).iloc[:,-1]

    print('df_corr:\n', df_corr)
    
    # for k in range(0, rt):
    #     print('result_' + str(k + 1) + '含有NaN数量：', df_corr['result_' + str(k + 1)].isna().sum())
    # print('\n')
    
    
    # print('df_corr.corr():\n', df_corr.corr())


    # 以向量形式得到两两列的相关系数
    df_corr = df_corr.corr()
    
    correlations = [
        df_corr.iloc[0, 1],  # col1 和 col2 的相关系数
        df_corr.iloc[0, 2],  # col1 和 col3 的相关系数
        df_corr.iloc[1, 2]   # col2 和 col3 的相关系数
    ]       
    
    print(correlations)
    # 检查是否有任意相关系数小于0.5
    if any(corr < 0.5 for corr in correlations):
        print("第"+str(i)+"个因子"+"结果不可靠,因为存在部分数据相关性较小\n")
        factor_reliable_list.append([i, False])
    else:
        print("第"+str(i)+"个因子"+"结果可靠\n")
        factor_reliable_list.append([i, True])

标准输出:
 Result saved to: F:/QuantData/factor_result/df_20109_1.feather

F:\\QuantData\\factor_result\\df_20109_1.feather
        SecuCode  TradeDay      RS_t
0      000001.SZ  20240603  0.000120
1      000001.SZ  20240604  0.000099
2      000001.SZ  20240605  0.000092
3      000001.SZ  20240606  0.000070
4      000001.SZ  20240607  0.000082
...          ...       ...       ...
98305  900957.SH  20240621  0.005438
98306  900957.SH  20240624  0.004505
98307  900957.SH  20240625  0.004590
98308  900957.SH  20240626  0.001496
98309  900957.SH  20240627  0.002272

[98310 rows x 3 columns]
含有NaN数量： 0
标准输出:
 Results saved to F:/QuantData/factor_result/df_20109_2.feather

F:\\QuantData\\factor_result\\df_20109_2.feather
        SecuCode  TradeDay      RS_t
0      000001.SZ  20240603  0.000120
1      000001.SZ  20240604  0.000099
2      000001.SZ  20240605  0.000092
3      000001.SZ  20240606  0.000070
4      000001.SZ  20240607  0.000082
...          ...       ...       ...
98305  900957.SH  20